In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

In [2]:
orig_url = 'https://drive.google.com/file/d/1uXK-rG07T1njpXvTNxNZcmJrVJQmfeRc/view?usp=sharing'
file_id = orig_url.split('/')[-2]
dwn_url = 'https://drive.google.com/uc?export=download&id=' + file_id

purchased = pd.read_excel(dwn_url, skiprows=1, names=['Desc', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
                                                      'Growth ratio'])
purchased

Desc    2011    2012    2013    2014    2015    2016    2017  \
0    Aantal (x 1000)    1198    1035    1008    1051     983     928     957   
1  Waarde (x € 1000)  879240  769159  797264  886605  899020  936829  976034   

      2018  Growth ratio  
0     1011          0.06  
1  1220291          0.25

In [3]:
#Overview
print(purchased.head())
print(purchased.shape)
# purchased.columns() # It looks like there is no such method for data frames.

                Desc    2011    2012    2013    2014    2015    2016    2017  \
0    Aantal (x 1000)    1198    1035    1008    1051     983     928     957   
1  Waarde (x € 1000)  879240  769159  797264  886605  899020  936829  976034   

      2018  Growth ratio  
0     1011          0.06  
1  1220291          0.25  
(2, 10)


In [4]:
#Checking the types
purchased.dtypes

Desc             object
2011              int64
2012              int64
2013              int64
2014              int64
2015              int64
2016              int64
2017              int64
2018              int64
Growth ratio    float64
dtype: object

In [5]:
#Adding the missing column. To be able to extrapolate the missing values later
purchased['2019']=""

In [6]:
# Calculating 2019 quantity based on growth ratio
purchased['2019'] = round(purchased['2018']*(1+purchased['Growth ratio']), 0).astype(int)
purchased

Desc    2011    2012    2013    2014    2015    2016    2017  \
0    Aantal (x 1000)    1198    1035    1008    1051     983     928     957   
1  Waarde (x € 1000)  879240  769159  797264  886605  899020  936829  976034   

      2018  Growth ratio     2019  
0     1011          0.06     1072  
1  1220291          0.25  1525364

In [7]:
#Taking out the columns and the row that we won't be needing
total_purchased = purchased.drop(['2011', '2012', '2013', '2014', '2015', 'Growth ratio'],axis=1)
total_purchased = total_purchased.loc[total_purchased['Desc'] == 'Aantal (x 1000)']

In [8]:
total_purchased['Desc'] = 'Purchased bikes NL'

In [9]:
total_purchased

Desc  2016  2017  2018  2019
0  Purchased bikes NL   928   957  1011  1072

In [10]:
#Displaying the correct unit to the Total_purchased df
total_purchased = total_purchased[['2016', '2017', '2018', '2019']].apply(lambda x: x*1000)
total_purchased.insert(0, 'Desc', 'Purchased bikes')


In [11]:
total_purchased 

Desc    2016    2017     2018     2019
0  Purchased bikes  928000  957000  1011000  1072000

In [12]:
# #THIS IS A TABLE I MADE WITH CALCULATIONS, but the tale was done elsewhere

# # Importing a separate table to be added to the already inserted 'purchase' one:
# url_1 = 'https://drive.google.com/file/d/1y1nWsBv4RHFIGLqAcgH84Iqq1a2rEWW-/view?usp=sharing'
# file_id = url_1.split('/')[-2]
# url = 'https://drive.google.com/uc?export=download&id=' + file_id
# purchased_ams = pd.read_excel(url, skiprows=1, nrows=6, names=['Desc', '2014', '2015', '2016','2017','2018','2019','tbd'])
# purchased_ams 
# # dropping unnecessary columns and rows:
# purchased_ams = purchased_ams.drop(['2014','2015','tbd'], axis =1)
# purchased_ams = purchased_ams.drop([0,2,3], axis =0)
# purchased_ams
#changing name in row to meaningful name
# purchased_ams['Desc'][1]='Purchased bikes AMS'
# purchased_ams

In [15]:
# We will create 3 dataframes, 1 where the calculations will be done, 2 with same avg amounts

#For the starting data I will add the poputaion AMs per year found in the final table in 'Population"'
pop_Ams = [{ 'Desc':'PopAMS','2016':821424,'2017':830788,'2018': 839459, '2019':848852}]
Pop_ams = pd.DataFrame(pop_Ams)
Pop_ams



Desc    2016    2017    2018    2019
0  PopAMS  821424  830788  839459  848852

In [17]:
pop_Nl = [{'Desc':'PopNL','2016':16931687,'2017':17031264,'2018': 17132933, '2019':17258355}]
Pop_nl = pd.DataFrame(pop_Nl)

Pop_nl

Desc      2016      2017      2018      2019
0  PopNL  16931687  17031264  17132933  17258355

In [22]:
# For the calculation I will do :(purchased per year/population NL)*Population AMS

Purchased_ams_total = (total_purchased._get_numeric_data() / Pop_nl._get_numeric_data())* Pop_ams._get_numeric_data()
Purchased_ams_total


2016          2017          2018          2019
0  45020.999502  46682.625318  49535.771196  52726.308156

In [23]:
Purchased_ams_total['Desc'] = 'Purchased bikes AMS'

In [25]:
column_order = ['Desc','2016','2017','2018','2019']
Purchased_ams_total = Purchased_ams_total[column_order ]

In [26]:
Purchased_ams_total

Desc          2016          2017          2018          2019
0  Purchased bikes AMS  45020.999502  46682.625318  49535.771196  52726.308156

In [28]:
F_AMS = [{ 'Desc':'Fetched bikes AMS','2016':6000,'2017':6000,'2018': 6000, '2019':6000}]
fetched_Ams = pd.DataFrame(F_AMS)
fetched_Ams

Desc  2016  2017  2018  2019
0  Fetched bikes AMS  6000  6000  6000  6000

In [29]:
F_NL = [{ 'Desc':'Fetched bikes NL','2016':15000,'2017':15000,'2018': 15000, '2019':15000}]
fetched_NL = pd.DataFrame(F_NL)
fetched_NL

Desc   2016   2017   2018   2019
0  Fetched bikes NL  15000  15000  15000  15000

In [30]:
#concatenating the tables
frames = [total_purchased,Purchased_ams_total,fetched_Ams,fetched_NL]
total_purchased = pd.concat(frames)

In [31]:
total_purchased

Desc           2016           2017          2018  \
0      Purchased bikes  928000.000000  957000.000000  1.011000e+06   
0  Purchased bikes AMS   45020.999502   46682.625318  4.953577e+04   
0    Fetched bikes AMS    6000.000000    6000.000000  6.000000e+03   
0     Fetched bikes NL   15000.000000   15000.000000  1.500000e+04   

           2019  
0  1.072000e+06  
0  5.272631e+04  
0  6.000000e+03  
0  1.500000e+04

In [ ]:
## Displaying correct unit in row 2:

# First I transpose the table, not to affect the rest of rows
# total_purchased.transpose()
# #Applying the formula for the concerned column
# total_purchased.iloc[1] = total_purchased.iloc[1].apply(lambda x: x*1000)

##Note: This actually affected the Desc. at index 1

# total_purchased
#Renaming Description at row index 1
# total_purchased['Desc'][1] = 'Purchased bikes AMS'
# total_purchased['']



In [32]:
# The data type is to be changed (for diplaying reasons)

convert_dict = {'Desc': object,
                '2016': int, 
                '2017': int,
                '2018': int,
                '2019': int,
               } 
  
total_purchased = total_purchased.astype(convert_dict) 

In [33]:
total_purchased

Desc    2016    2017     2018     2019
0      Purchased bikes  928000  957000  1011000  1072000
0  Purchased bikes AMS   45020   46682    49535    52726
0    Fetched bikes AMS    6000    6000     6000     6000
0     Fetched bikes NL   15000   15000    15000    15000